**Artificial Inteligence (CS550)**
<br>
Date: **13 April 2021**
<br>


Title: **Seminar 20**

Speaker: **Dr. Shota Tsiskaridze**


Bibliography: 
<br>
[1] **Chapter 17**, Jeremy Howard & Sylvain Gugger, Deep Learning for Coders with fastai & PyTorch, O'Reilly Media, Inc., 2020

In [1]:
#hide
#!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

# A Neural Net from the Foundations

## The Forward and Backward Passes

To train a model, we will need to compute all the gradients of a given loss with respect to its parameters, which is known as the *backward pass*. 

The *forward pass* is where we compute the output of the model on a given input, based on the matrix products. 

As we define our first neural net, we will also delve into the problem of properly initializing the weights, which is crucial for making training start properly.

### Defining and Initializing a Layer

We will take the example of a two-layer neural net first. 

As we've seen, one layer can be expressed as `y = x @ w + b`, with `x` our inputs, `y` our outputs, `w` the weights of the layer 
<br> (which is of size number of inputs by number of neurons if we don't transpose like before), and `b` is the bias vector:

In [60]:
def lin(x, w, b): return x @ w + b

We can stack the second layer on top of the first, but since mathematically the composition of two linear operations is another linear operation, this only makes sense if we put something nonlinear in the middle, called an activation function. 

As mentioned before, in deep learning applications the activation function most commonly used is a ReLU, which returns the maximum of `x` and `0`. 

We won't actually train our model today, so we'll use random tensors for our inputs and targets. 

Let's say our inputs are 200 vectors of size 100, which we group into one batch, and our targets are 200 random floats:

In [61]:
x = torch.randn(200, 100)
y = torch.randn(200)

For our two-layer model we will need two weight matrices and two bias vectors. 

Let's say we have a hidden size of 50 and the output size is 1 (for one of our inputs, the corresponding output is one float in this toy example). 

We initialize the weights randomly and the bias at zero:

In [62]:
w1 = torch.randn(100,50)
b1 = torch.zeros(50)
w2 = torch.randn(50,1)
b2 = torch.zeros(1)

Then the result of our first layer is simply:

In [63]:
l1 = lin(x, w1, b1)
l1.shape

torch.Size([200, 50])

Note that this formula works with our batch of inputs, and returns a batch of hidden state: 
<br>`l1` is a matrix of size 200 (our batch size) by 50 (our hidden size).

There is a problem with the way our model was initialized, however. 

To understand it, we need to look at the mean and standard deviation (std) of `l1`:

In [64]:
l1.mean(), l1.std()

(tensor(-0.2733), tensor(10.1770))

The mean is close to zero, which is understandable since both our input and weight matrices have means close to zero. 

But the standard deviation, which represents how far away our activations go from the mean, went from 1 to 10. 

This is a really big problem because that's with just one layer. 

Modern neural nets can have hundred of layers, so if each of them multiplies the scale of our activations by 10.

By the end of the last layer we won't have numbers representable by a computer.

Indeed, if we make just 50 multiplications between `x` and random matrices of size 100×100, we'll have:

In [65]:
x = torch.randn(200, 100)

for i in range(50): 
    x = x @ torch.randn(100,100)

x[0:5,0:5]

tensor([[nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan]])

The result is `nan`s everywhere. 

So maybe the scale of our matrix was too big, and we need to have smaller weights? 

But if we use too small weights, we will have the opposite problem - the scale of our activations will go from 1 to 0.1, and after 100 layers we'll be left with zeros everywhere:

In [66]:
x = torch.randn(200, 100)

for i in range(50): 
    x = x @ (torch.randn(100,100) * 0.01)
    
x[0:5,0:5]

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

So we have to scale our weight matrices exactly right so that the standard deviation of our activations stays at 1. 

We can compute the exact value to use mathematically, as illustrated by Xavier Glorot and Yoshua Bengio in:

- ["Understanding the Difficulty of Training Deep Feedforward Neural Networks"](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf). 

The right scale for a given layer is $\frac{1}{\sqrt{n_{in}}}$, where $n_{in}$ represents the number of inputs.

In our case, if we have 100 inputs, we should scale our weight matrices by 0.1:

In [67]:
x = torch.randn(200, 100)

for i in range(50): 
    x = x @ (torch.randn(100,100) * 0.1)
    
x[0:5,0:5]

tensor([[ 0.3490,  0.2760,  0.7100, -0.5662,  0.0849],
        [ 0.0242, -0.1556,  0.0356,  0.5157,  0.1024],
        [ 0.4934,  0.1478,  0.6660, -0.8834,  0.2357],
        [-0.3257,  0.1484, -0.3587, -0.6842,  0.0687],
        [-0.4128,  0.4351,  0.3021, -0.3969, -0.0684]])

Finally some numbers that are neither zeros nor `nan`s! 

Notice how stable the scale of our activations is, even after those 50 fake layers:

In [68]:
x.std()

tensor(0.5306)

If you play a little bit with the value for scale you'll notice that even a slight variation from 0.1 will get you either to very small or very large numbers, so initializing the weights properly is extremely important. 

Let's go back to our neural net. Since we messed a bit with our inputs, we need to redefine them:

In [54]:
x = torch.randn(200, 100)
y = torch.randn(200)

And for our weights, we'll use the right scale, which is known as *Xavier initialization* (or *Glorot initialization*):

In [70]:
from math import sqrt

w1 = torch.randn(100,50) / sqrt(100)
b1 = torch.zeros(50)
w2 = torch.randn(50,1) / sqrt(50)
b2 = torch.zeros(1)

Now if we compute the result of the first layer, we can check that the mean and standard deviation are under control:

In [71]:
l1 = lin(x, w1, b1)
l1.mean(),l1.std()

(tensor(0.0092), tensor(1.0159))

Very good!

Now we need to go through a ReLU, so let's define one. 

A ReLU removes the negatives and replaces them with zeros, which is another way of saying it clamps our tensor at zero:

In [72]:
def relu(x): return x.clamp_min(0.)

We pass our activations through this:

In [73]:
l2 = relu(l1)
l2.mean(),l2.std()

(tensor(0.4076), tensor(0.5961))

And we're back to square one: the mean of our activations has gone to 0.4 

This is understandable since we removed the negatives and the std went down to 0.58. 

So like before, after a few layers we will probably wind up with zeros:

In [74]:
x = torch.randn(200, 100)

for i in range(50): 
    x = relu(x @ (torch.randn(100,100) * 0.1))
    
x[0:5,0:5]

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 2.1702e-10],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 2.7826e-10],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 2.5480e-10],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 3.9613e-10]])

This means our initialization wasn't right. Why? 

At the time Glorot and Bengio wrote their article, the popular activation in a neural net was the hyperbolic tangent (tanh, which is the one they used), and that initialization doesn't account for our ReLU. 

Fortunately, someone else has done the math for us and computed the right scale for us to use. 

In ["Delving Deep into Rectifiers: Surpassing Human-Level Performance"](https://arxiv.org/abs/1502.01852) (which we've seen before—it's the article that introduced the ResNet), Kaiming He et al. show that we should use the following scale instead: $\sqrt{\frac{2}{n_{in}}}$, where $n_{in}$ is the number of inputs of our model. 

Let's see what this gives us:

In [75]:
x = torch.randn(200, 100)

for i in range(50): 
    x = relu(x @ (torch.randn(100,100) * sqrt(2/100)))
    
x[0:5,0:5]

tensor([[0.0979, 0.2917, 0.0000, 0.2479, 0.7577],
        [0.0685, 0.0611, 0.0000, 0.0000, 0.8962],
        [0.1678, 0.0553, 0.0000, 0.0000, 1.2590],
        [0.1115, 0.0784, 0.0000, 0.0000, 1.0734],
        [0.0481, 0.0263, 0.0000, 0.0099, 0.4877]])

That's better: our numbers aren't all zeroed this time. 

So let's go back to the definition of our neural net and use this initialization (which is named *Kaiming initialization* or *He initialization*):

In [61]:
x = torch.randn(200, 100)
y = torch.randn(200)

In [62]:
w1 = torch.randn(100,50) * sqrt(2 / 100)
b1 = torch.zeros(50)
w2 = torch.randn(50,1) * sqrt(2 / 50)
b2 = torch.zeros(1)

Let's look at the scale of our activations after going through the first linear layer and ReLU:

In [63]:
l1 = lin(x, w1, b1)
l2 = relu(l1)
l2.mean(), l2.std()

(tensor(0.5656), tensor(0.8275))

Much better! 

Now that our weights are properly initialized, we can define our whole model:

In [64]:
def model(x):
    l1 = lin(x, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

This is the forward pass. 

Now all that's left to do is to compare our output to the labels we have (random numbers, in this example) with a loss function. 

In this case, we will use the mean squared error. 

It's a toy problem, and this is the easiest loss function to use for what is next, computing the gradients.

The only subtlety is that our outputs and targets don't have exactly the same shape—after going though the model, we get an output like this:

In [65]:
out = model(x)
out.shape

torch.Size([200, 1])

To get rid of this trailing 1 dimension, we use the `squeeze` function:

In [66]:
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

And now we are ready to compute our loss:

In [67]:
loss = mse(out, y)

That's all for the forward pass - let's now look at the gradients.

### Gradients and the Backward Pass

We've seen that PyTorch computes all the gradients we need with a magic call to `loss.backward`, but let's explore what's happening behind the scenes.

Now comes the part where we need to compute the gradients of the loss with respect to all the weights of our model: `w1`, `b1`, `w2`, and `b2`. 

For this, we will need a bit of math - specifically the **chain rule**. 

This is the rule of calculus that guides how we can compute the derivative of a composed function:

$$(g \circ f)'(x) = g'(f(x)) f'(x)$$

- **Jeremy**: 

  I find this notation very hard to wrap my head around, so instead I like to think of it as
  
  if `y = g(u)` and `u=f(x)`, then `dy/dx = dy/du * du/dx`.

  The two notations mean the same thing, so use whatever works for you.

Our loss is a big composition of different functions: mean squared error (which is in turn the composition of a mean and a power of two), the second linear layer, a ReLU and the first linear layer. 

For instance, if we want the gradients of the loss with respect to `b2` and our loss is defined by:

```
loss = mse(out,y) = mse(lin(l2, w2, b2), y)
```

The chain rule tells us that we have:
$$\frac{\text{d} loss}{\text{d} b_{2}} = \frac{\text{d} loss}{\text{d} out} \times \frac{\text{d} out}{\text{d} b_{2}} = \frac{\text{d}}{\text{d} out} mse(out, y) \times \frac{\text{d}}{\text{d} b_{2}} lin(l_{2}, w_{2}, b_{2})$$

To compute the gradients of the loss with respect to $b_{2}$, we first need the gradients of the loss with respect to our output $out$. 

It's the same if we want the gradients of the loss with respect to $w_{2}$. 

Then, to get the gradients of the loss with respect to $b_{1}$ or $w_{1}$, we will need the gradients of the loss with respect to $l_{1}$, which in turn requires the gradients of the loss with respect to $l_{2}$, which will need the gradients of the loss with respect to $out$.

So to compute all the gradients we need for the update, we need to begin from the output of the model and work our way *backward*, one layer after the other - which is why this step is known as *backpropagation*. 

We can automate it by having each function we implemented (`relu`, `mse`, `lin`) provide its backward step: that is, how to derive the gradients of the loss with respect to the input(s) from the gradients of the loss with respect to the output.

Here we populate those gradients in an attribute of each tensor, a bit like PyTorch does with `.grad`. 

The first are the gradients of the loss with respect to the output of our model (which is the input of the loss function). We undo the `squeeze` we did in `mse`, then we use the formula that gives us the derivative of $x^{2}$: $2x$. 

The derivative of the mean is just $1/n$ where $n$ is the number of elements in our input:

In [68]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

For the gradients of the ReLU and our linear layer, we use the gradients of the loss with respect to the output (in `out.g`) and apply the chain rule to compute the gradients of the loss with respect to the output (in `inp.g`). The chain rule tells us that `inp.g = relu'(inp) * out.g`. The derivative of `relu` is either 0 (when inputs are negative) or 1 (when inputs are positive), so this gives us:

In [69]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

The scheme is the same to compute the gradients of the loss with respect to the inputs, weights, and bias in the linear layer:

In [70]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = inp.t() @ out.g
    b.g = out.g.sum(0)

We won't linger on the mathematical formulas that define them since they're not important for our purposes, but do check out Khan Academy's excellent calculus lessons if you're interested in this topic.

### Sidebar: SymPy

SymPy is a library for symbolic computation that is extremely useful library when working with calculus. Per the [documentation](https://docs.sympy.org/latest/tutorial/intro.html):

> : Symbolic computation deals with the computation of mathematical objects symbolically. This means that the mathematical objects are represented exactly, not approximately, and mathematical expressions with unevaluated variables are left in symbolic form.

To do symbolic computation, we first define a *symbol*, and then do a computation, like so:

In [71]:
from sympy import symbols,diff
sx,sy = symbols('sx sy')
diff(sx**2, sx)

2*sx

Here, SymPy has taken the derivative of `x**2` for us! 

It can take the derivative of complicated compound expressions, simplify and factor equations, and much more. 

There's really not much reason for anyone to do calculus manually nowadays—for calculating gradients, PyTorch does it for us, and for showing the equations, SymPy does it for us!

### End sidebar

Once we have have defined those functions, we can use them to write the backward pass. 

Since each gradient is automatically populated in the right tensor, we don't need to store the results of those `_grad` functions anywhere—we just need to execute them in the reverse order of the forward pass, to make sure that in each function `out.g` exists:

In [72]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

And now we can access the gradients of our model parameters in `w1.g`, `b1.g`, `w2.g`, and `b2.g`.

We have successfully defined our model—now let's make it a bit more like a PyTorch module.

### Refactoring the Model

The three functions we used have two associated functions: a forward pass and a backward pass. 

Instead of writing them separately, we can create a class to wrap them together. 

That class can also store the inputs and outputs for the backward pass. 

This way, we will just have to call `backward`:

In [73]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

`__call__` is a magic name in Python that will make our class callable. 

This is what will be executed when we type `y = Relu()(x)`. 

We can do the same for our linear layer and the MSE loss:

In [74]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [75]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        x = (self.inp.squeeze()-self.targ).unsqueeze(-1)
        self.inp.g = 2.*x/self.targ.shape[0]

Then we can put everything in a model that we initiate with our tensors `w1`, `b1`, `w2`, `b2`:

In [76]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

What is really nice about this refactoring and registering things as layers of our model is that the forward and backward passes are now really easy to write. If we want to instantiate our model, we just need to write:

In [77]:
model = Model(w1, b1, w2, b2)

The forward pass can then be executed with:

In [78]:
loss = model(x, y)

And the backward pass with:

In [79]:
model.backward()

### Going to PyTorch

The  `Lin`, `Mse` and `Relu` classes we wrote have a lot in common, so we could make them all inherit from the same base class:

In [80]:
class LayerFunction():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self):  raise Exception('not implemented')
    def bwd(self):      raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

Then we just need to implement `forward` and `bwd` in each of our subclasses:

In [81]:
class Relu(LayerFunction):
    def forward(self, inp): return inp.clamp_min(0.)
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [82]:
class Lin(LayerFunction):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ self.out.g
        self.b.g = out.g.sum(0)

In [83]:
class Mse(LayerFunction):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): 
        inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

The rest of our model can be the same as before. 

This is getting closer and closer to what PyTorch does. 

Each basic function we need to differentiate is written as a `torch.autograd.Function` object that has a `forward` and a `backward` method. 

PyTorch will then keep trace of any computation we do to be able to properly run the backward pass, unless we set the `requires_grad` attribute of our tensors to `False`.

Writing one of these is (almost) as easy as writing our original classes. 

The difference is that we choose what to save and what to put in a context variable (so that we make sure we don't save anything we don't need), and we return the gradients in the `backward` pass. 

It's very rare to have to write your own `Function` but if you ever need something exotic or want to mess with the gradients of a regular function, here is how to write one:

In [84]:
from torch.autograd import Function

class MyRelu(Function):
    @staticmethod
    def forward(ctx, i):
        result = i.clamp_min(0.)
        ctx.save_for_backward(i)
        return result
    
    @staticmethod
    def backward(ctx, grad_output):
        i, = ctx.saved_tensors
        return grad_output * (i>0).float()

The structure used to build a more complex model that takes advantage of those `Function`s is a `torch.nn.Module`. 

This is the base structure for all models, and all the neural nets you have seen up until now inherited from that class. 

It mostly helps to register all the trainable parameters, which as we've seen can be used in the training loop.

To implement an `nn.Module` you just need to:

- Make sure the superclass `__init__` is called first when you initialize it.
- Define any parameters of the model as attributes with `nn.Parameter`.
- Define a `forward` function that returns the output of your model.

As an example, here is the linear layer from scratch:

In [85]:
import torch.nn as nn

class LinearLayer(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(n_out, n_in) * sqrt(2/n_in))
        self.bias = nn.Parameter(torch.zeros(n_out))
    
    def forward(self, x): return x @ self.weight.t() + self.bias

As you see, this class automatically keeps track of what parameters have been defined:

In [86]:
lin = LinearLayer(10,2)
p1,p2 = lin.parameters()
p1.shape,p2.shape

(torch.Size([2, 10]), torch.Size([2]))

It is thanks to this feature of `nn.Module` that we can just say `opt.step()` and have an optimizer loop through the parameters and update each one.

Note that in PyTorch, the weights are stored as an `n_out x n_in` matrix, which is why we have the transpose in the forward pass.

By using the linear layer from PyTorch (which uses the Kaiming initialization as well), the model we have been building up during this chapter can be written like this:

In [87]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out))
        self.loss = mse
        
    def forward(self, x, targ): return self.loss(self.layers(x).squeeze(), targ)

fastai provides its own variant of `Module` that is identical to `nn.Module`, but doesn't require you to call `super().__init__()` (it does that for you automatically):

In [89]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        self.layers = nn.Sequential(
            nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out))
        self.loss = mse
        
    def forward(self, x, targ): return self.loss(self.layers(x).squeeze(), targ)

In the last chapter, we will start from such a model and see how to build a training loop from scratch and refactor it to what we've been using in previous chapters.

## Conclusion

In this chapter we explored the foundations of deep learning, beginning with matrix multiplication and moving on to implementing the forward and backward passes of a neural net from scratch. We then refactored our code to show how PyTorch works beneath the hood.

Here are a few things to remember:

- A neural net is basically a bunch of matrix multiplications with nonlinearities in between.
- Python is slow, so to write fast code we have to vectorize it and take advantage of techniques such as elementwise arithmetic and broadcasting.
- Two tensors are broadcastable if the dimensions starting from the end and going backward match (if they are the same, or one of them is 1). To make tensors broadcastable, we may need to add dimensions of size 1 with `unsqueeze` or a `None` index.
- Properly initializing a neural net is crucial to get training started. Kaiming initialization should be used when we have ReLU nonlinearities.
- The backward pass is the chain rule applied multiple times, computing the gradients from the output of our model and going back, one layer at a time.
- When subclassing `nn.Module` (if not using fastai's `Module`) we have to call the superclass `__init__` method in our `__init__` method and we have to define a `forward` function that takes an input and returns the desired result.

## Questionnaire

1. Write the Python code to implement a single neuron.
1. Write the Python code to implement ReLU.
1. Write the Python code for a dense layer in terms of matrix multiplication.
1. Write the Python code for a dense layer in plain Python (that is, with list comprehensions and functionality built into Python).
1. What is the "hidden size" of a layer?
1. What does the `t` method do in PyTorch?
1. Why is matrix multiplication written in plain Python very slow?
1. In `matmul`, why is `ac==br`?
1. In Jupyter Notebook, how do you measure the time taken for a single cell to execute?
1. What is "elementwise arithmetic"?
1. Write the PyTorch code to test whether every element of `a` is greater than the corresponding element of `b`.
1. What is a rank-0 tensor? How do you convert it to a plain Python data type?
1. What does this return, and why? `tensor([1,2]) + tensor([1])`
1. What does this return, and why? `tensor([1,2]) + tensor([1,2,3])`
1. How does elementwise arithmetic help us speed up `matmul`?
1. What are the broadcasting rules?
1. What is `expand_as`? Show an example of how it can be used to match the results of broadcasting.
1. How does `unsqueeze` help us to solve certain broadcasting problems?
1. How can we use indexing to do the same operation as `unsqueeze`?
1. How do we show the actual contents of the memory used for a tensor?
1. When adding a vector of size 3 to a matrix of size 3×3, are the elements of the vector added to each row or each column of the matrix? (Be sure to check your answer by running this code in a notebook.)
1. Do broadcasting and `expand_as` result in increased memory use? Why or why not?
1. Implement `matmul` using Einstein summation.
1. What does a repeated index letter represent on the left-hand side of einsum?
1. What are the three rules of Einstein summation notation? Why?
1. What are the forward pass and backward pass of a neural network?
1. Why do we need to store some of the activations calculated for intermediate layers in the forward pass?
1. What is the downside of having activations with a standard deviation too far away from 1?
1. How can weight initialization help avoid this problem?
1. What is the formula to initialize weights such that we get a standard deviation of 1 for a plain linear layer, and for a linear layer followed by ReLU?
1. Why do we sometimes have to use the `squeeze` method in loss functions?
1. What does the argument to the `squeeze` method do? Why might it be important to include this argument, even though PyTorch does not require it?
1. What is the "chain rule"? Show the equation in either of the two forms presented in this chapter.
1. Show how to calculate the gradients of `mse(lin(l2, w2, b2), y)` using the chain rule.
1. What is the gradient of ReLU? Show it in math or code. (You shouldn't need to commit this to memory—try to figure it using your knowledge of the shape of the function.)
1. In what order do we need to call the `*_grad` functions in the backward pass? Why?
1. What is `__call__`?
1. What methods must we implement when writing a `torch.autograd.Function`?
1. Write `nn.Linear` from scratch, and test it works.
1. What is the difference between `nn.Module` and fastai's `Module`?